In [ ]:
import torch
from NLTM.interlingua_model.model.modelling_mbart_modified_posembedding import MBartForConditionalGeneration
from NLTM.interlingua_model.data_handling.datacollater import DataCollatorForSeq2Seq

In [ ]:
import wandb
wandb.init(project="NLTM_POS", entity="divyanshuusingh")

In [ ]:
torch.cuda.is_available()

In [ ]:
from transformers import (AutoTokenizer,
                          AutoModelForSeq2SeqLM,
                          #DataCollatorForSeq2Seq,
                          Seq2SeqTrainingArguments,
                          Seq2SeqTrainer)


In [ ]:
model_name = "ai4bharat/IndicBART"
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, use_fast=False, keep_accents=True)
#

In [ ]:
model = MBartForConditionalGeneration.from_pretrained(pretrained_model_name_or_path = "/home/ece/utkarsh/interlingua-model/")

In [ ]:
#model.__dict__

# seeing class MBartEncoder(MBartPreTrainedModel) 

In [ ]:
model # yeah

In [ ]:
#model._modules['model']._modules['encoder']

In [ ]:
model._modules['model']._modules['encoder'].embed_positions

In [ ]:
""" 
lets write mbartposembedding---

Q1. what input is model taking and then giving to function 

"""

# import dataset cdac

In [ ]:
from datasets import load_from_disk

In [ ]:
tok_data = load_from_disk('/home/ece/utkarsh/interlingua-model/IndicBART/data_from_cdac/MKB_NIOS_11k')

In [ ]:
tok_data["train"][0]

# training

In [ ]:
from NLTM.interlingua_model.trainer.trainer_seq2seq import Seq2SeqTrainer
from NLTM.interlingua_model.trainer import trainer
from NLTM.interlingua_model.data_handling.datacollater import DataCollatorForSeq2Seq

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)  

In [ ]:
from transformers import (AutoTokenizer,
                          AutoModelForSeq2SeqLM,
                          #DataCollatorForSeq2Seq,
                          Seq2SeqTrainingArguments
                          )
import numpy as np

In [ ]:
batch_size = 16

In [ ]:
args = Seq2SeqTrainingArguments(
            'translation',
            evaluation_strategy='epoch',
            learning_rate=0.001,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            weight_decay=0.0001,
            #save_total_limit=15,
            num_train_epochs=30,
            #label_names = ["pos"],
            predict_with_generate=True,
            report_to="wandb")

In [ ]:
args.__dict__

In [ ]:
tokenized_datasets = tok_data

In [ ]:
from datasets import load_metric

In [ ]:
metric = load_metric('sacrebleu')

In [ ]:
def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [[label.strip()] for label in labels]

        return preds, labels

def compute_metrics(eval_preds):
        #print("compute_met called")
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
        result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {'bleu': result['score']}

        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result['gen_len'] = np.mean(prediction_lens)
        result = {k: round(v, 4) for k, v in result.items()}
        #print("compute-met-end")
        return result


In [ ]:
trainer = Seq2SeqTrainer(
              model,
              args,
              train_dataset=tokenized_datasets['train'],
              eval_dataset=tokenized_datasets['test'],
              data_collator=data_collator,
              tokenizer=tokenizer,
              compute_metrics=compute_metrics)  



In [ ]:
trainer.train()

## evaluate after training

In [ ]:
max_target_length = 128


In [ ]:
trainer.evaluate()

predict_dataset = tokenized_datasets['test']
predict_results = trainer.predict(predict_dataset,
                                    metric_key_prefix='predict',
                                    max_length=max_target_length)

predictions = tokenizer.batch_decode(predict_results.predictions,
                                        skip_special_tokens=True,
                                        clean_up_tokenization_spaces=True)

predictions = [pred.strip() for pred in predictions]

# input_sentences = tokenizer.batch_decode(tokenized_datasets['test']['input_ids'], skip_special_tokens=True)
# output_sentences = tokenizer.batch_decode(tokenized_datasets['test']['labels'], skip_special_tokens=True)


In [ ]:
predictions

In [ ]:
import pandas as pd

In [ ]:
source_lang = 'Hindi'
target_lang = 'Sanskrit'

In [ ]:
language_code = {'Hindi':'hi', 'Sanskrit':'sa', 'Kannada':'kn'}
#s_lang = language_code[FLAGS.source_lang.capitalize()]
#t_lang = language_code[FLAGS.target_lang.capitalize()]
s_lang = 'hi'
t_lang = 'sa'

In [ ]:

# if FLAGS.target_lang.capitalize() == 'Kannada':
#     kannada_sentences = []
#     for sentence in predictions:
#         sentence = UnicodeIndicTransliterator.transliterate(sentence, 'hi', 'kn')
#         kannada_sentences.append(sentence)      

predictions_data = pd.DataFrame({f'{source_lang}':tokenized_datasets['test'][source_lang.capitalize()], 
                                    f'{target_lang}':tokenized_datasets['test'][target_lang.capitalize()],
                                    'predictions':  predictions})

predictions_data.to_csv(f'{source_lang}_{target_lang}_predictions.csv')                                                                                                                                                                       
